In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Defaults

# System settings
systmpfs = '/tmp'

# Optimizer settings
num_workers = 2
timeout = 60

In [24]:
import os
import tempfile
import subprocess

from multiprocessing import Pool

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
sns.set(rc={'figure.figsize': (15, 10), 'figure.dpi' : 250})

import nevergrad as ng
from SALib.sample import saltelli
from SALib.analyze import sobol
import geotopy as gtp

In [4]:
class GEOtopSensitivityRun(gtp.GEOtop):

    def preprocess(self, working_dir, *args, **kwargs):

        settings = {**self.settings, **kwargs}
        
        geotop_inpts_path = os.path.join(working_dir, 'geotop.inpts')
        os.remove(geotop_inpts_path)
        with open(geotop_inpts_path, 'w') as geotop_inpts:
            geotop_inpts.write("! GEOtop input file written by GEOtoPy\n")
            for key, value in settings.items():
                geotop_inpts.write(gtp.print_setting(key, value))    
        
    def postprocess(self, working_dir):
        
        liq_path = os.path.join(working_dir, 'theta_liq.txt')
        liq = pd.read_csv(liq_path, na_values=['-9999'],
                          usecols=[0, 6, 7, 8], 
                          skiprows=1,
                          header=0, 
                          names=["date", 20, 50, 200],
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        
        ice_path = os.path.join(working_dir, 'theta_ice.txt')
        ice = pd.read_csv(ice_path, na_values=['-9999'], 
                          usecols=[0, 6, 7, 8], 
                          skiprows=1,
                          header=0, 
                          names=["date", 20, 50, 200],
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        
        sim = ice + liq
        
        obs_path = os.path.join(working_dir, 'obs.csv')
        obs = pd.read_csv(obs_path, 
                          usecols=[0, 6, 7, 8], 
                          header=0, 
                          names=["date", 20, 50, 200], 
                          na_values=['-9999'], 
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        obs = obs
        
        return obs, sim

    
def diff_metric(obs, sim):
    
    diff = (obs - sim)
    diff_squared = diff * diff
    obs_squared = obs * obs
    
    return np.sqrt(diff_squared.mean().sum() / obs_squared.mean().sum())


def loss(*args, **kwargs):
    
    model = GEOtopSensitivityRun('data/inputs', 
                                 exe='../../geotop/build/geotop',
                                 run_args={'check': True, 'capture_output': True, 'timeout': timeout})
    
    with tempfile.TemporaryDirectory(dir=systmpfs) as tmpdir:
        try:
            output = model.eval(tmpdir, *args, **kwargs)
        except subprocess.CalledProcessError:
            return np.nan
        except subprocess.TimeoutExpired:
            return np.nan
    
    return diff_metric(*output)

In [5]:
problem = {
    'num_vars': 2,
    'names': ['LSAI', 'InitSoilPressure'],
    'bounds': [[0.0, 10.0],
               [-5e3, 0.0]]
}

In [21]:
param_values = saltelli.sample(problem, 2)
settings = [{key: value for key, value in zip(problem['names'], values)} for values in param_values]

In [45]:
def loss_wrapper(kwargs):
    return loss(**kwargs)

with Pool(processes=num_workers) as pool:
    Y = pool.map(loss_wrapper, settings)

In [50]:
sobol.analyze(problem, np.array(Y), print_to_console=True);

Parameter S1 S1_conf ST ST_conf
LSAI 0.000000 0.000000 0.000000 0.000000
InitSoilPressure 1.985695 0.127616 1.999740 0.000256

Parameter_1 Parameter_2 S2 S2_conf
LSAI InitSoilPressure 0.000000 0.000000
